# N-Queen Problem using Evolutionary Algorithm
## Kiarash Mokhtari - 9830032

In [70]:
import random
import time

### 1. First we define Initialize Population function which randomly generates individuals.

In [71]:
def initialize_population(population_size: int, board_size: int) -> list:
    population = []
    for _ in range(population_size):
        individual = random.sample(range(board_size), board_size)
        population.append(individual)
    
    return population

In [72]:
# Inorder to test the function we can run below code:
def print_matrix(matrix):
    for row in matrix:
        print(row)
        
population_size = 10
board_size = 8

sample_population = initialize_population(population_size, board_size)
print_matrix(sample_population)

[2, 0, 4, 5, 7, 6, 3, 1]
[6, 5, 4, 7, 0, 1, 3, 2]
[5, 4, 2, 7, 0, 3, 6, 1]
[0, 3, 5, 4, 6, 1, 7, 2]
[4, 3, 2, 6, 0, 1, 5, 7]
[4, 6, 5, 3, 0, 2, 1, 7]
[5, 2, 0, 4, 1, 3, 6, 7]
[1, 5, 4, 6, 2, 7, 0, 3]
[1, 7, 5, 0, 3, 6, 2, 4]
[0, 3, 4, 6, 7, 5, 1, 2]


### 2. we define Evaluate Fitness fucntion which evaluates the fitness of each individual in the population.

In [86]:
def evaluate_fitness(individual):
    conflicts = 0
    size = len(individual)

    for i in range(size):
        for j in range(i+1, size):
            if individual[i] == individual[j] or abs(individual[i] - individual[j]) == abs(i - j):
                conflicts += 1

    fitness = 1 / (conflicts + 1)

    return fitness

In [87]:
# Inorder to test the function we can run below code:
for individual in sample_population:
    print(evaluate_fitness(individual))

0.16666666666666666
0.1
0.2
0.25
0.1111111111111111
0.1111111111111111
0.2
0.125
0.16666666666666666
0.16666666666666666


### 3. we define Selection fucntion which selects individuals from the population based on their fitness values. .
#### here I implement 3 approaches and then I will compare them with each other

In [88]:
# 1 approach
def tournament_selection(population, tournament_size):
    selected = []
    
    for _ in range(len(population)):
        sub_population = random.sample(population, tournament_size)
        winner = max(sub_population, key=evaluate_fitness)
        selected.append(winner)
    
    return selected

In [89]:
# 2 approach
def roulette_wheel_selection(population):
    total_fitness = 0
    selection_probs = []
    selected = []

    for individual in population:
        total_fitness += evaluate_fitness(individual)
    
    for individual in population:
        selection_probs.append(evaluate_fitness(individual) / total_fitness)

    for _ in range(len(population)):
        cumulative_prob = 0.0
        r = random.random()

        for i, individual in enumerate(population):
            cumulative_prob += selection_probs[i]
            if r <= cumulative_prob:
                selected.append(individual)
                break

    return selected

In [90]:
# 3 approach
def rank_based_selection(population):
    selection_probs = []
    population_size = len(population)
    ranks = list(range(1, population_size + 1))
    total_ranks = sum(ranks)
    
    for rank in ranks:
        selection_probs.append(rank / total_ranks)

    selected = []
    for _ in range(population_size):
        r = random.random()
        cumulative_prob = 0.0

        for i, individual in enumerate(population):
            cumulative_prob += selection_probs[i]
            if r <= cumulative_prob:
                selected.append(individual)
                break

    return selected

### 4. we define Crossover fucntion which performs crossover or recombination between selected individuals to create offspring.


In [91]:
def crossover(parent1, parent2):
    size = len(parent1)
    crossover_point = random.randint(1, size - 1)

    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return crossover_point, child1, child2

In [92]:
# Inorder to test the function we can run below code:
parent1 = [0, 3, 6, 4, 7, 1, 5, 2]
parent2 = [4, 1, 3, 6, 2, 0, 7, 5]

print("Children before Crossover:")
print(child1)
print(child2)
print("========================================================")

crossover_point, child1, child2 = crossover(parent1, parent2)
print("Children after Crossover:")
print(child1)
print(child2)
print("Crossover Point: ", crossover_point)

Children before Crossover:
[0, 3, 6, 4, 7, 1, 7, 5]
[4, 1, 3, 6, 2, 0, 5, 2]
Children after Crossover:
[0, 3, 3, 6, 2, 0, 7, 5]
[4, 1, 6, 4, 7, 1, 5, 2]
Crossover Point:  2


### 5. we define Mutation fucntion which adds diversity to the population and helps explore different regions of the solution space.

In [93]:
def mutation(individual, mutation_rate):
    size = len(individual)
    mutated_individual = individual.copy()

    for i in range(size):
        if random.random() < mutation_rate:
            new_position = random.randint(0, size - 1)
            mutated_individual[i] = new_position

    return mutated_individual

In [94]:
# Inorder to test the function we can run below code:
individual = [0, 3, 6, 4, 7, 1, 5, 2]
mutation_rate = 0.1

mutated_individual = mutation(individual, mutation_rate)
print(mutated_individual)

[0, 3, 6, 4, 7, 1, 3, 2]


### 6. we define Replacement fucntion which selects individuals from the parent population and the offspring population to form the next generation.

In [95]:
def replacement(parent_population, offspring_population, population_size):
    combined_population = parent_population + offspring_population

    combined_population.sort(key=evaluate_fitness, reverse=True)

    next_generation = combined_population[:population_size]

    return next_generation

### 7. we define Termination fucntion which determines when to stop the algorithm.

In [96]:
def termination_condition(population, max_fitness):
    for individual in population:
        if evaluate_fitness(individual) >= max_fitness:
            return True
    return False

### 8. Main function tournament_selection function

In [104]:
def solve_n_queen(population_size, board_size, max_generations, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0) and current_generation < max_generations:
        parent_population = tournament_selection(population, 3)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            crossover_point, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    return best_individual, best_fitness

In [105]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())
        
# Example for 8-Queen
population_size = 100
board_size = 8
max_generations = 1000
mutation_rate = 0.1

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("8-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

print("\n")

# Example for 16-Queen
population_size = 200
board_size = 16
max_generations = 2000
mutation_rate = 0.05

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("16-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

8-Queen Problem:
Best Individual: [4, 1, 3, 6, 2, 7, 5, 0]
- - - - Q - - -
- Q - - - - - -
- - - Q - - - -
- - - - - - Q -
- - Q - - - - -
- - - - - - - Q
- - - - - Q - -
Q - - - - - - -
Best Fitness: 1.0
Execution Time: 0.10819506645202637 seconds


16-Queen Problem:
Best Individual: [2, 15, 13, 3, 8, 4, 1, 6, 0, 14, 11, 9, 5, 7, 10, 12]
- - Q - - - - - - - - - - - - -
- - - - - - - - - - - - - - - Q
- - - - - - - - - - - - - Q - -
- - - Q - - - - - - - - - - - -
- - - - - - - - Q - - - - - - -
- - - - Q - - - - - - - - - - -
- Q - - - - - - - - - - - - - -
- - - - - - Q - - - - - - - - -
Q - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - Q -
- - - - - - - - - - - Q - - - -
- - - - - - - - - Q - - - - - -
- - - - - Q - - - - - - - - - -
- - - - - - - Q - - - - - - - -
- - - - - - - - - - Q - - - - -
- - - - - - - - - - - - Q - - -
Best Fitness: 0.3333333333333333
Execution Time: 56.022629499435425 seconds


### 9. Main function with roulette_wheel_selection

In [110]:
def solve_n_queen(population_size, board_size, max_generations, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0) and current_generation < max_generations:
        parent_population = roulette_wheel_selection(population)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            crossover_point, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    return best_individual, best_fitness

In [111]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())
        
# Example for 8-Queen
population_size = 100
board_size = 8
max_generations = 1000
mutation_rate = 0.1

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("8-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

print("\n")

# Example for 16-Queen
population_size = 200
board_size = 16
max_generations = 2000
mutation_rate = 0.05

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("16-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

8-Queen Problem:
Best Individual: [5, 3, 1, 7, 4, 6, 0, 2]
- - - - - Q - -
- - - Q - - - -
- Q - - - - - -
- - - - - - - Q
- - - - Q - - -
- - - - - - Q -
Q - - - - - - -
- - Q - - - - -
Best Fitness: 1.0
Execution Time: 0.004334926605224609 seconds


16-Queen Problem:
Best Individual: [13, 10, 7, 9, 14, 4, 0, 15, 12, 1, 5, 8, 2, 11, 3, 6]
- - - - - - - - - - - - - Q - -
- - - - - - - - - - Q - - - - -
- - - - - - - Q - - - - - - - -
- - - - - - - - - Q - - - - - -
- - - - - - - - - - - - - - Q -
- - - - Q - - - - - - - - - - -
Q - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - - Q
- - - - - - - - - - - - Q - - -
- Q - - - - - - - - - - - - - -
- - - - - Q - - - - - - - - - -
- - - - - - - - Q - - - - - - -
- - Q - - - - - - - - - - - - -
- - - - - - - - - - - Q - - - -
- - - Q - - - - - - - - - - - -
- - - - - - Q - - - - - - - - -
Best Fitness: 0.5
Execution Time: 51.831241607666016 seconds


### 10. Main function with rank_based_selection function

In [108]:
def solve_n_queen(population_size, board_size, max_generations, mutation_rate):
    population = initialize_population(population_size, board_size)
    current_generation = 0

    while not termination_condition(population, 1.0) and current_generation < max_generations:
        parent_population = rank_based_selection(population)
        offspring_population = []

        while len(offspring_population) < population_size:
            parent1, parent2 = random.sample(parent_population, 2)
            crossover_point, child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, mutation_rate)
            mutated_child2 = mutation(child2, mutation_rate)
            offspring_population.extend([mutated_child1, mutated_child2])

        population = replacement(parent_population, offspring_population, population_size)
        current_generation += 1

    best_individual = max(population, key=evaluate_fitness)
    best_fitness = evaluate_fitness(best_individual)
    return best_individual, best_fitness

In [109]:
def print_individual(individual):
    board_size = len(individual)

    for row in range(board_size):
        line = ""
        for col in range(board_size):
            if individual[row] == col:
                line += "Q "
            else:
                line += "- "
        print(line.strip())
        
# Example for 8-Queen
population_size = 100
board_size = 8
max_generations = 1000
mutation_rate = 0.1

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("8-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

print("\n")

# Example for 16-Queen
population_size = 200
board_size = 16
max_generations = 2000
mutation_rate = 0.05

start_time = time.time()
best_individual, best_fitness = solve_n_queen(population_size, board_size, max_generations, mutation_rate)
end_time = time.time()

execution_time = end_time - start_time

print("16-Queen Problem:")
print("Best Individual:", best_individual)
print_individual(best_individual)
print("Best Fitness:", best_fitness)
print("Execution Time:", execution_time, "seconds")

8-Queen Problem:
Best Individual: [1, 3, 5, 0, 2, 6, 0, 7]
- Q - - - - - -
- - - Q - - - -
- - - - - Q - -
Q - - - - - - -
- - Q - - - - -
- - - - - - Q -
Q - - - - - - -
- - - - - - - Q
Best Fitness: 0.25
Execution Time: 2.51774525642395 seconds


16-Queen Problem:
Best Individual: [5, 4, 6, 9, 2, 15, 0, 14, 7, 8, 12, 1, 3, 0, 10, 13]
- - - - - Q - - - - - - - - - -
- - - - Q - - - - - - - - - - -
- - - - - - Q - - - - - - - - -
- - - - - - - - - Q - - - - - -
- - Q - - - - - - - - - - - - -
- - - - - - - - - - - - - - - Q
Q - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - Q -
- - - - - - - Q - - - - - - - -
- - - - - - - - Q - - - - - - -
- - - - - - - - - - - - Q - - -
- Q - - - - - - - - - - - - - -
- - - Q - - - - - - - - - - - -
Q - - - - - - - - - - - - - - -
- - - - - - - - - - Q - - - - -
- - - - - - - - - - - - - Q - -
Best Fitness: 0.125
Execution Time: 30.874314546585083 seconds
